In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diseases-and-symptoms-dataset/Final_Augmented_dataset_Diseases_and_Symptoms.csv
/kaggle/input/disease-and-symptoms-dataset/DiseaseAndSymptoms.csv
/kaggle/input/disease-and-symptoms-dataset/Disease precaution.csv


In [2]:
!pip install scikit-learn transformers torch torchmetrics pandas numpy

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

import pickle
import os


2025-06-18 08:14:20.688394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750234460.734875     791 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750234460.748884     791 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import os

print("Files in first dataset:")
print(os.listdir("/kaggle/input/diseases-and-symptoms-dataset"))

print("\nFiles in second dataset:")
print(os.listdir("/kaggle/input/disease-and-symptoms-dataset"))


Files in first dataset:
['Final_Augmented_dataset_Diseases_and_Symptoms.csv']

Files in second dataset:
['DiseaseAndSymptoms.csv', 'Disease precaution.csv']


In [5]:
# Corrected paths for reading datasets
df1 = pd.read_csv('/kaggle/input/diseases-and-symptoms-dataset/Final_Augmented_dataset_Diseases_and_Symptoms.csv')
df2 = pd.read_csv('/kaggle/input/disease-and-symptoms-dataset/DiseaseAndSymptoms.csv')

# Check their structure
df1.head(), df2.head()


(         diseases  anxiety and nervousness  depression  shortness of breath  \
 0  panic disorder                        1           0                    1   
 1  panic disorder                        0           0                    1   
 2  panic disorder                        1           1                    1   
 3  panic disorder                        1           0                    0   
 4  panic disorder                        1           1                    0   
 
    depressive or psychotic symptoms  sharp chest pain  dizziness  insomnia  \
 0                                 1                 0          0         0   
 1                                 1                 0          1         1   
 2                                 1                 0          1         1   
 3                                 1                 0          1         1   
 4                                 0                 0          0         1   
 
    abnormal involuntary movements  chest 

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# X = All symptoms
X = df1.drop(columns=['diseases'])

# y = Target label
y = df1['diseases']

# Encode disease labels to numbers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Confirm
print("Shape of X:", X.shape)
print("Unique diseases:", len(label_encoder.classes_))


Shape of X: (246945, 377)
Unique diseases: 773


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Model Architecture
model = Sequential([
    Dense(512, activation='relu', input_shape=(X.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-18 08:14:35.194857: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/50
4939/4939 - 57s - 12ms/step - accuracy: 0.6780 - loss: 1.3563 - val_accuracy: 0.8378 - val_loss: 0.4775
Epoch 2/50
4939/4939 - 51s - 10ms/step - accuracy: 0.7862 - loss: 0.7230 - val_accuracy: 0.8399 - val_loss: 0.4392
Epoch 3/50
4939/4939 - 51s - 10ms/step - accuracy: 0.8010 - loss: 0.6569 - val_accuracy: 0.8450 - val_loss: 0.4310
Epoch 4/50
4939/4939 - 50s - 10ms/step - accuracy: 0.8080 - loss: 0.6266 - val_accuracy: 0.8483 - val_loss: 0.4166
Epoch 5/50
4939/4939 - 50s - 10ms/step - accuracy: 0.8147 - loss: 0.5985 - val_accuracy: 0.8482 - val_loss: 0.4149
Epoch 6/50
4939/4939 - 50s - 10ms/step - accuracy: 0.8187 - loss: 0.5786 - val_accuracy: 0.8524 - val_loss: 0.3945
Epoch 7/50
4939/4939 - 51s - 10ms/step - accuracy: 0.8229 - loss: 0.5598 - val_accuracy: 0.8513 - val_loss: 0.4036
Epoch 8/50
4939/4939 - 50s - 10ms/step - accuracy: 0.8250 - loss: 0.5500 - val_accuracy: 0.8528 - val_loss: 0.3961
Epoch 9/50
4939/4939 - 50s - 10ms/step - accuracy: 0.8278 - loss: 0.5380 - val_a

In [8]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Predict on test set
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))

# Get only present class labels in y_test
present_classes = np.unique(y_test)
present_class_names = label_encoder.inverse_transform(present_classes)

# Print classification report safely
print("\nClassification Report:\n", classification_report(
    y_test,
    y_pred,
    labels=present_classes,
    target_names=present_class_names,
    zero_division=0  # Avoids warnings for undefined precision/recall
))


1544/1544 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
Accuracy: 0.8606369839437932

Classification Report:
                                                           precision    recall  f1-score   support

                               abdominal aortic aneurysm       0.74      1.00      0.85        20
                                        abdominal hernia       0.94      0.99      0.96        81
                                         abscess of nose       0.83      0.86      0.85        58
                                     abscess of the lung       1.00      0.83      0.91         6
                                  abscess of the pharynx       0.87      0.98      0.93        63
                                    acanthosis nigricans       0.62      0.83      0.71         6
                                               acariasis       1.00      1.00      1.00         5
                                               achalasia       0.62      1.00      0.77        20
                    

In [9]:
import pickle

# Save model
model.save('/kaggle/working/BIMARIBOT_Model.h5')

# Save label encoder
with open('/kaggle/working/BIMARIBOT_LabelEncoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Save symptoms list for frontend input mapping
with open('/kaggle/working/BIMARIBOT_SymptomsList.pkl', 'wb') as f:
    pickle.dump(list(X.columns), f)


In [10]:
import numpy as np
import pickle
from tensorflow.keras.models import load_model

# Load everything
model = load_model('/kaggle/working/BIMARIBOT_Model.h5')

with open('/kaggle/working/BIMARIBOT_LabelEncoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

with open('/kaggle/working/BIMARIBOT_SymptomsList.pkl', 'rb') as f:
    symptom_list = pickle.load(f)  # All symptoms used during training


def predict_top_diseases(symptoms_input, top_k=5):
    input_symptoms = [s.strip().lower() for s in symptoms_input.split(',')]
    input_vector = np.zeros((1, len(symptom_list)))
    
    for idx, symptom in enumerate(symptom_list):
        if symptom.lower() in input_symptoms:
            input_vector[0, idx] = 1

    probs = model.predict(input_vector)[0]
    top_indices = np.argsort(probs)[-top_k:][::-1]
    results = [(label_encoder.inverse_transform([i])[0], round(probs[i]*100, 2)) for i in top_indices]

    return results



In [11]:
user_input = "running nose,headache, cough,sore throat,weakness"
predictions = predict_top_diseases(user_input, top_k=5)
for disease, prob in predictions:
    print(f"{disease} - {prob}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
seasonal allergies (hay fever) - 34.98%
mononucleosis - 12.78%
pneumonia - 10.92%
atelectasis - 10.02%
nose disorder - 5.82%


In [12]:
import pandas as pd

# Load all datasets
augmented_df = pd.read_csv('/kaggle/input/diseases-and-symptoms-dataset/Final_Augmented_dataset_Diseases_and_Symptoms.csv')
symptom_df = pd.read_csv('/kaggle/input/disease-and-symptoms-dataset/DiseaseAndSymptoms.csv')
precaution_df = pd.read_csv('/kaggle/input/disease-and-symptoms-dataset/Disease precaution.csv')

# Preview
print("📊 Augmented Dataset:\n", augmented_df.head())
print("📊 Symptom Dataset:\n", symptom_df.head())
print("📊 Precaution Dataset:\n", precaution_df.head())


📊 Augmented Dataset:
          diseases  anxiety and nervousness  depression  shortness of breath  \
0  panic disorder                        1           0                    1   
1  panic disorder                        0           0                    1   
2  panic disorder                        1           1                    1   
3  panic disorder                        1           0                    0   
4  panic disorder                        1           1                    0   

   depressive or psychotic symptoms  sharp chest pain  dizziness  insomnia  \
0                                 1                 0          0         0   
1                                 1                 0          1         1   
2                                 1                 0          1         1   
3                                 1                 0          1         1   
4                                 0                 0          0         1   

   abnormal involuntary movements 

In [13]:
print(augmented_df.columns.tolist())

['diseases', 'anxiety and nervousness', 'depression', 'shortness of breath', 'depressive or psychotic symptoms', 'sharp chest pain', 'dizziness', 'insomnia', 'abnormal involuntary movements', 'chest tightness', 'palpitations', 'irregular heartbeat', 'breathing fast', 'hoarse voice', 'sore throat', 'difficulty speaking', 'cough', 'nasal congestion', 'throat swelling', 'diminished hearing', 'lump in throat', 'throat feels tight', 'difficulty in swallowing', 'skin swelling', 'retention of urine', 'groin mass', 'leg pain', 'hip pain', 'suprapubic pain', 'blood in stool', 'lack of growth', 'emotional symptoms', 'elbow weakness', 'back weakness', 'pus in sputum', 'symptoms of the scrotum and testes', 'swelling of scrotum', 'pain in testicles', 'flatulence', 'pus draining from ear', 'jaundice', 'mass in scrotum', 'white discharge from eye', 'irritable infant', 'abusing alcohol', 'fainting', 'hostile behavior', 'drug abuse', 'sharp abdominal pain', 'feeling ill', 'vomiting', 'headache', 'nause

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import joblib
from fuzzywuzzy import process
import warnings
import os
import time
warnings.filterwarnings('ignore')

# Start timer
start_time = time.time()

# Create Kaggle working directory
os.makedirs('/kaggle/working', exist_ok=True)
print(f"Setup time: {time.time() - start_time:.2f} seconds")

# Load datasets
load_start = time.time()
augmented_df = pd.read_csv('/kaggle/input/diseases-and-symptoms-dataset/Final_Augmented_dataset_Diseases_and_Symptoms.csv')
precaution_df = pd.read_csv('/kaggle/input/disease-and-symptoms-dataset/Disease precaution.csv')
print(f"Data loading time: {time.time() - load_start:.2f} seconds")
print(f"Dataset size: {augmented_df.shape}")

# Optional: Sample dataset to reduce size (uncomment if needed)
# sample_start = time.time()
# augmented_df = augmented_df.sample(frac=0.5, random_state=42)
# print(f"Sampling time: {time.time() - sample_start:.2f} seconds")

# Filter rare classes
filter_start = time.time()
min_samples = 5
class_counts = augmented_df['diseases'].value_counts()
valid_classes = class_counts[class_counts >= min_samples].index
augmented_df = augmented_df[augmented_df['diseases'].isin(valid_classes)]
print(f"Filtering time: {time.time() - filter_start:.2f} seconds")

# Check class distribution
print("Class Distribution after filtering (Top 5 and Bottom 5):")
print(class_counts.head(5))
print("\nClasses with fewer than 5 samples (removed):")
print(class_counts[class_counts < 5])

# Preprocess data
preprocess_start = time.time()
X = augmented_df.drop('diseases', axis=1)
y = augmented_df['diseases']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(f"Number of unique diseases after filtering: {len(label_encoder.classes_)}")
print(f"Preprocessing time: {time.time() - preprocess_start:.2f} seconds")

# Split data
split_start = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
print(f"Data splitting time: {time.time() - split_start:.2f} seconds")

# Initialize XGBoost with GPU
train_start = time.time()
model = XGBClassifier(
    n_estimators=100,  # Reduced for speed
    max_depth=3,       # Reduced for speed
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='mlogloss',
    tree_method='gpu_hist'  # GPU acceleration
)
model.fit(X_train, y_train)
print(f"Training time: {time.time() - train_start:.2f} seconds")

# Evaluate model
eval_start = time.time()
test_accuracy = model.score(X_test, y_test)
print(f"Test Set Accuracy: {test_accuracy:.2f}")
print(f"Evaluation time: {time.time() - eval_start:.2f} seconds")

# Save model and encoder
save_start = time.time()
joblib.dump(model, '/kaggle/working/model.joblib')
joblib.dump(label_encoder, '/kaggle/working/label_encoder.joblib')
print("Model and Label Encoder saved to /kaggle/working/")
print(f"Saving time: {time.time() - save_start:.2f} seconds")

# Fuzzy matching for symptoms
def match_symptoms(user_symptoms, valid_symptoms, threshold=85):
    matched_symptoms = []
    for symptom in user_symptoms:
        match = process.extractOne(symptom, valid_symptoms, score_cutoff=threshold)
        if match:
            matched_symptoms.append(match[0])
            print(f"Matched '{symptom}' to '{match[0]}' (score: {match[1]})")
        else:
            print(f"Warning: No match found for symptom '{symptom}'")
    return matched_symptoms

# Predict disease and precautions
def predict_disease_and_precautions(symptoms_list):
    valid_symptoms = X.columns.tolist()
    matched_symptoms = match_symptoms(symptoms_list, valid_symptoms)
    symptom_vector = pd.Series(0, index=X.columns)
    for symptom in matched_symptoms:
        symptom_vector[symptom] = 1
    symptom_vector = symptom_vector.values.reshape(1, -1)
    predicted_label = model.predict(symptom_vector)[0]
    predicted_disease = label_encoder.inverse_transform([predicted_label])[0]
    precaution_diseases = precaution_df['Disease'].tolist()
    match = process.extractOne(predicted_disease, precaution_diseases, score_cutoff=70)
    if match:
        matched_disease = match[0]
        print(f"Matched disease '{predicted_disease}' to '{matched_disease}' (score: {match[1]})")
    else:
        matched_disease = predicted_disease
        print(f"No precaution match found for '{predicted_disease}'")
    precautions = precaution_df[precaution_df['Disease'] == matched_disease]
    precaution_list = precautions.iloc[0][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].dropna().tolist() if not precautions.empty else ["No precautions found for this disease."]
    return {'Predicted Disease': predicted_disease, 'Precautions': precaution_list}

# Save valid symptoms
symptoms_start = time.time()
with open('/kaggle/working/valid_symptoms.txt', 'w') as f:
    f.write('\n'.join(X.columns.tolist()))
print("Valid symptoms saved to /kaggle/working/valid_symptoms.txt")
print("Sample of 10 symptoms:", X.columns[:10].tolist())
print(f"Symptoms saving time: {time.time() - symptoms_start:.2f} seconds")

# Example prediction
predict_start = time.time()
user_symptoms = ['anxiety and nervousness', 'shortness of breath', 'chest tightness']
result = predict_disease_and_precautions(user_symptoms)
print("\nPrediction Result:")
print(f"Predicted Disease: {result['Predicted Disease']}")
print("Precautions:")
for i, precaution in enumerate(result['Precautions'], 1):
    print(f"{i}. {precaution}")
print(f"Prediction time: {time.time() - predict_start:.2f} seconds")

print(f"Total runtime: {(time.time() - start_time)/60:.2f} minutes")

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Setup time: 0.00 seconds
Data loading time: 6.75 seconds
Dataset size: (246945, 378)
Filtering time: 0.32 seconds
Class Distribution after filtering (Top 5 and Bottom 5):
diseases
cystitis                          1219
vulvodynia                        1218
nose disorder                     1218
complex regional pain syndrome    1217
spondylosis                       1216
Name: count, dtype: int64

Classes with fewer than 5 samples (removed):
diseases
open wound of the shoulder                            4
syndrome of inappropriate secretion of adh (siadh)    4
intussusception                                       4
amyloidosis                                           4
meckel diverticulum                                   4
dumping syndrome                                      4
open wound of the hand                                4
scurvy                                                4
oral leukoplakia                                      4
open wound of the finger                

In [2]:
import pandas as pd
import joblib
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from fuzzywuzzy import process
import numpy as np
import re
import os
import time
import warnings
warnings.filterwarnings('ignore')

# Start timer
start_time = time.time()

# Clear /kaggle/working/
os.system('rm -rf /kaggle/working/*')
os.makedirs('/kaggle/working', exist_ok=True)
print("Cleared /kaggle/working/ directory")

# Check disk space
print("Disk space before training:")
os.system('df -h /kaggle/working/')

# Load and preprocess dataset
augmented_df = pd.read_csv('/kaggle/input/diseases-and-symptoms-dataset/Final_Augmented_dataset_Diseases_and_Symptoms.csv')
precaution_df = pd.read_csv('/kaggle/input/disease-and-symptoms-dataset/Disease precaution.csv')

# Filter rare classes
min_samples = 10
class_counts = augmented_df['diseases'].value_counts()
valid_classes = class_counts[class_counts >= min_samples].index
augmented_df = augmented_df[augmented_df['diseases'].isin(valid_classes)]
print(f"Number of unique diseases after filtering: {len(valid_classes)}")

# Optional: Sample dataset (uncomment if memory issues)
# augmented_df = augmented_df.sample(frac=0.5, random_state=42)
print(f"Dataset size: {augmented_df.shape}")

# Sanitize feature names
def sanitize_name(name):
    name = re.sub(r'[^a-zA-Z0-9\s]', '_', name)
    name = re.sub(r'\s+', '_', name)
    return name.strip('_')

# Preprocess data
X = augmented_df.drop('diseases', axis=1)
X.columns = [sanitize_name(col) for col in X.columns]
y = augmented_df['diseases']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train LightGBM
try:
    model = lgb.LGBMClassifier(
        n_estimators=100,
        max_depth=3,
        num_leaves=15,
        learning_rate=0.1,
        min_data_in_leaf=50,
        random_state=42,
        device='cpu'  # Force CPU for stability
    )
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='multi_logloss',
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)]
    )
    accuracy = model.score(X_test, y_test)
    print(f"Test Set Accuracy: {accuracy:.2f}")
except Exception as e:
    print(f"LightGBM training failed: {e}")
    print("Falling back to XGBoost...")
    from xgboost import XGBClassifier
    model = XGBClassifier(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,
        random_state=42,
        eval_metric='mlogloss'
    )
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f"XGBoost Test Set Accuracy: {accuracy:.2f}")

# Save model and encoder
try:
    joblib.dump(model, '/kaggle/working/model_lightgbm.joblib')
    joblib.dump(label_encoder, '/kaggle/working/label_encoder_lightgbm.joblib')
    print("Model and Label Encoder saved to /kaggle/working/")
    # Verify files exist
    if os.path.exists('/kaggle/working/model_lightgbm.joblib'):
        print("Confirmed: model_lightgbm.joblib saved")
    if os.path.exists('/kaggle/working/label_encoder_lightgbm.joblib'):
        print("Confirmed: label_encoder_lightgbm.joblib saved")
except Exception as e:
    print(f"Failed to save files: {e}")

print("Disk space after saving:")
os.system('df -h /kaggle/working/')

# Prediction function
valid_symptoms = X.columns.tolist()
original_symptoms = augmented_df.drop('diseases', axis=1).columns.tolist()

def match_symptoms(user_symptoms, valid_symptoms, original_symptoms, threshold=85):
    matched_symptoms = []
    for symptom in user_symptoms:
        match = process.extractOne(symptom, original_symptoms, score_cutoff=threshold)
        if match:
            matched_symptom = sanitize_name(match[0])
            matched_symptoms.append(matched_symptom)
            print(f"Matched '{symptom}' to '{match[0]}' (sanitized: '{matched_symptom}', score: {match[1]})")
        else:
            print(f"Warning: No match found for symptom '{symptom}'")
    return matched_symptoms

def predict_disease_and_precautions(symptoms_list):
    matched_symptoms = match_symptoms(symptoms_list, valid_symptoms, original_symptoms)
    if not matched_symptoms:
        return {'Predicted Disease': 'None', 'Precautions': ['No valid symptoms provided.'], 'Confidence': 0.0, 'Top Predictions': []}
    
    symptom_vector = pd.Series(0, index=valid_symptoms)
    for symptom in matched_symptoms:
        if symptom in symptom_vector.index:
            symptom_vector[symptom] = 1
        else:
            print(f"Warning: Sanitized symptom '{symptom}' not in feature set")
    
    symptom_vector = symptom_vector.values.reshape(1, -1)
    
    predicted_label = model.predict(symptom_vector)[0]
    probabilities = model.predict_proba(symptom_vector)[0]
    confidence = probabilities[predicted_label]
    predicted_disease = label_encoder.inverse_transform([predicted_label])[0]
    
    top_indices = np.argsort(probabilities)[-3:][::-1]
    top_predictions = [
        (label_encoder.inverse_transform([idx])[0], probabilities[idx])
        for idx in top_indices
    ]
    
    precaution_diseases = precaution_df['Disease'].tolist()
    valid_diseases = augmented_df['diseases'].unique().tolist()
    top_matches = process.extract(predicted_disease, precaution_diseases, limit=5)
    print(f"Top 5 precaution matches for '{predicted_disease}':")
    for match in top_matches:
        print(f"  - {match[0]} (score: {match[1]})")
    
    match = process.extractOne(predicted_disease, precaution_diseases, score_cutoff=70)
    if match and match[0] in valid_diseases:
        matched_disease = match[0]
        print(f"Matched disease '{predicted_disease}' to '{matched_disease}' (score: {match[1]})")
    else:
        matched_disease = predicted_disease
        print(f"No valid precaution match found for '{predicted_disease}'")
    precautions = precaution_df[precaution_df['Disease'] == matched_disease]
    precaution_list = precautions.iloc[0][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].dropna().tolist() if not precautions.empty else ["No precautions found for this disease."]
    
    return {
        'Predicted Disease': predicted_disease,
        'Precautions': precaution_list,
        'Confidence': confidence,
        'Top Predictions': top_predictions
    }

# Test predictions
test_symptoms = [
    ['vomiting', 'diarrhea', 'weakness'],
    ['headache', 'fever', 'cough'],
    ['anxiety and nervousness', 'shortness of breath', 'chest tightness']
]

for symptoms in test_symptoms:
    print(f"\nTesting symptoms: {symptoms}")
    result = predict_disease_and_precautions(symptoms)
    print(f"Predicted Disease: {result['Predicted Disease']}")
    print(f"Confidence: {result['Confidence']:.2f}")
    print("Top 3 Predictions:")
    for disease, prob in result['Top Predictions']:
        print(f"  - {disease}: {prob:.2f}")
    print("Precautions:")
    for i, precaution in enumerate(result['Precautions'], 1):
        print(f"{i}. {precaution}")

print(f"Total runtime: {(time.time() - start_time)/60:.2f} minutes")

Cleared /kaggle/working/ directory
Disk space before training:
Filesystem      Size  Used Avail Use% Mounted on
/dev/loop1       20G   76K   20G   1% /kaggle/working
Number of unique diseases after filtering: 677
Dataset size: (246512, 378)
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.196296 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 618
[LightGBM] [Info] Number of data points in the train set: 197209, number of used features: 309
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Start training from s

In [3]:
import pandas as pd
import joblib
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from fuzzywuzzy import process
import numpy as np
import re
import os
import time
import warnings
warnings.filterwarnings('ignore')

# Start timer
start_time = time.time()

# Optional: Clear /kaggle/working/ (commented out to preserve existing files)
# os.system('rm -rf /kaggle/working/*')
os.makedirs('/kaggle/working', exist_ok=True)
print("Ensured /kaggle/working/ directory exists")

# Check disk space
print("Disk space before training:")
os.system('df -h /kaggle/working/')

# Load and preprocess dataset
augmented_df = pd.read_csv('/kaggle/input/diseases-and-symptoms-dataset/Final_Augmented_dataset_Diseases_and_Symptoms.csv')
precaution_df = pd.read_csv('/kaggle/input/disease-and-symptoms-dataset/Disease precaution.csv')

# Filter rare classes
min_samples = 10
class_counts = augmented_df['diseases'].value_counts()
valid_classes = class_counts[class_counts >= min_samples].index
augmented_df = augmented_df[augmented_df['diseases'].isin(valid_classes)]
print(f"Number of unique diseases after filtering: {len(valid_classes)}")
print(f"Dataset size: {augmented_df.shape}")

# Sanitize feature names
def sanitize_name(name):
    name = re.sub(r'[^a-zA-Z0-9\s]', '_', name)
    name = re.sub(r'\s+', '_', name)
    return name.strip('_')

# Preprocess data
X = augmented_df.drop('diseases', axis=1)
X.columns = [sanitize_name(col) for col in X.columns]
y = augmented_df['diseases']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train LightGBM
try:
    model = lgb.LGBMClassifier(
        n_estimators=500,
        max_depth=5,
        num_leaves=31,
        learning_rate=0.05,
        min_data_in_leaf=20,
        class_weight='balanced',
        random_state=42,
        device='cpu',
        force_row_wise=True
    )
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='multi_logloss',
        callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=True)]
    )
    accuracy = model.score(X_test, y_test)
    print(f"Test Set Accuracy: {accuracy:.2f}")
except Exception as e:
    print(f"LightGBM training failed: {e}")
    print("Falling back to XGBoost...")
    from xgboost import XGBClassifier
    model = XGBClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.05,
        random_state=42,
        eval_metric='mlogloss'
    )
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f"XGBoost Test Set Accuracy: {accuracy:.2f}")

# Save model and encoder with new names
try:
    joblib.dump(model, '/kaggle/working/model_lightgbm_v2.joblib')
    joblib.dump(label_encoder, '/kaggle/working/label_encoder_lightgbm_v2.joblib')
    print("Model and Label Encoder saved to /kaggle/working/")
    if os.path.exists('/kaggle/working/model_lightgbm_v2.joblib'):
        print("Confirmed: model_lightgbm_v2.joblib saved")
    if os.path.exists('/kaggle/working/label_encoder_lightgbm_v2.joblib'):
        print("Confirmed: label_encoder_lightgbm_v2.joblib saved")
except Exception as e:
    print(f"Failed to save files: {e}")

print("Disk space after saving:")
os.system('df -h /kaggle/working/')

# Prediction function
valid_symptoms = X.columns.tolist()
original_symptoms = augmented_df.drop('diseases', axis=1).columns.tolist()

def match_symptoms(user_symptoms, valid_symptoms, original_symptoms, threshold=85):
    matched_symptoms = []
    for symptom in user_symptoms:
        match = process.extractOne(symptom, original_symptoms, score_cutoff=threshold)
        if match:
            matched_symptom = sanitize_name(match[0])
            matched_symptoms.append(matched_symptom)
            print(f"Matched '{symptom}' to '{match[0]}' (sanitized: '{matched_symptom}', score: {match[1]})")
        else:
            print(f"Warning: No match found for symptom '{symptom}'")
    return matched_symptoms

def predict_disease_and_precautions(symptoms_list):
    matched_symptoms = match_symptoms(symptoms_list, valid_symptoms, original_symptoms)
    if not matched_symptoms:
        return {'Predicted Disease': 'None', 'Precautions': ['No valid symptoms provided.'], 'Confidence': 0.0, 'Top Predictions': []}
    
    symptom_vector = pd.Series(0, index=valid_symptoms)
    for symptom in matched_symptoms:
        if symptom in symptom_vector.index:
            symptom_vector[symptom] = 1
        else:
            print(f"Warning: Sanitized symptom '{symptom}' not in feature set")
    
    symptom_vector = symptom_vector.values.reshape(1, -1)
    
    predicted_label = model.predict(symptom_vector)[0]
    probabilities = model.predict_proba(symptom_vector)[0]
    confidence = probabilities[predicted_label]
    predicted_disease = label_encoder.inverse_transform([predicted_label])[0]
    
    top_indices = np.argsort(probabilities)[-3:][::-1]
    top_predictions = [
        (label_encoder.inverse_transform([idx])[0], probabilities[idx])
        for idx in top_indices
    ]
    
    precaution_diseases = precaution_df['Disease'].tolist()
    top_matches = process.extract(predicted_disease, precaution_diseases, limit=5)
    print(f"Top 5 precaution matches for '{predicted_disease}':")
    for match in top_matches:
        print(f"  - {match[0]} (score: {match[1]})")
    
    match = process.extractOne(predicted_disease, precaution_diseases, score_cutoff=65)
    if match:
        matched_disease = match[0]
        print(f"Matched disease '{predicted_disease}' to '{matched_disease}' (score: {match[1]})")
    else:
        matched_disease = predicted_disease
        print(f"No precaution match found for '{predicted_disease}'")
    precautions = precaution_df[precaution_df['Disease'] == matched_disease]
    precaution_list = precautions.iloc[0][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].dropna().tolist() if not precautions.empty else ["No precautions found for this disease."]
    
    return {
        'Predicted Disease': predicted_disease,
        'Precautions': precaution_list,
        'Confidence': confidence,
        'Top Predictions': top_predictions
    }

# Test predictions
test_symptoms = [
    ['vomiting', 'diarrhea', 'weakness'],
    ['headache', 'fever', 'cough'],
    ['anxiety and nervousness', 'shortness of breath', 'chest tightness']
]

for symptoms in test_symptoms:
    print(f"\nTesting symptoms: {symptoms}")
    result = predict_disease_and_precautions(symptoms)
    print(f"Predicted Disease: {result['Predicted Disease']}")
    print(f"Confidence: {result['Confidence']:.2f}")
    print("Top 3 Predictions:")
    for disease, prob in result['Top Predictions']:
        print(f"  - {disease}: {prob:.2f}")
    print("Precautions:")
    for i, precaution in enumerate(result['Precautions'], 1):
        print(f"{i}. {precaution}")

print(f"Total runtime: {(time.time() - start_time)/60:.2f} minutes")

Ensured /kaggle/working/ directory exists
Disk space before training:
Filesystem      Size  Used Avail Use% Mounted on
/dev/loop1       20G  760K   20G   1% /kaggle/working
Number of unique diseases after filtering: 677
Dataset size: (246512, 378)
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 197209, number of used features: 317
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Start training from score -6.517671
[LightGBM] [Info] Start training from score -6.517671
[LightGBM] [Info] Start training from score -6.517671
[LightGBM] [Info] Start training from score -6.517671
[LightGBM] [Info] Start training from scor

In [4]:
import pandas as pd
import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from fuzzywuzzy import process
import numpy as np
import re
import os
import time
from sklearn.feature_selection import VarianceThreshold
import warnings
warnings.filterwarnings('ignore')

# Start timer
start_time = time.time()

# Ensure /kaggle/working/ exists
os.makedirs('/kaggle/working', exist_ok=True)
print("Ensured /kaggle/working/ directory exists")

# Check disk space
print("Disk space before training:")
os.system('df -h /kaggle/working/')

# Load and preprocess dataset
augmented_df = pd.read_csv('/kaggle/input/diseases-and-symptoms-dataset/Final_Augmented_dataset_Diseases_and_Symptoms.csv')
precaution_df = pd.read_csv('/kaggle/input/disease-and-symptoms-dataset/Disease precaution.csv')

# Filter rare classes
min_samples = 10
class_counts = augmented_df['diseases'].value_counts()
valid_classes = class_counts[class_counts >= min_samples].index
augmented_df = augmented_df[augmented_df['diseases'].isin(valid_classes)]
print(f"Number of unique diseases after filtering: {len(valid_classes)}")
print(f"Dataset size: {augmented_df.shape}")

# Sanitize feature names
def sanitize_name(name):
    name = re.sub(r'[^a-zA-Z0-9\s]', '_', name)
    name = re.sub(r'\s+', '_', name)
    return name.strip('_')

# Preprocess data
X = augmented_df.drop('diseases', axis=1)
X.columns = [sanitize_name(col) for col in X.columns]
y = augmented_df['diseases']

# Feature selection: Keep top 100 features by variance
selector = VarianceThreshold(threshold=0.01)
X_selected = selector.fit_transform(X)
selected_features = X.columns[selector.get_support()].tolist()
X = pd.DataFrame(X_selected, columns=selected_features)
print(f"Selected {len(selected_features)} features after variance thresholding")

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train XGBoost
try:
    model = XGBClassifier(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='mlogloss'
    )
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f"Test Set Accuracy: {accuracy:.2f}")
except Exception as e:
    print(f"XGBoost training failed: {e}")
    raise

# Save model and encoder
try:
    joblib.dump(model, '/kaggle/working/model_xgboost_v3.joblib')
    joblib.dump(label_encoder, '/kaggle/working/label_encoder_xgboost_v3.joblib')
    print("Model and Label Encoder saved to /kaggle/working/")
    if os.path.exists('/kaggle/working/model_xgboost_v3.joblib'):
        print("Confirmed: model_xgboost_v3.joblib saved")
    if os.path.exists('/kaggle/working/label_encoder_xgboost_v3.joblib'):
        print("Confirmed: label_encoder_xgboost_v3.joblib saved")
except Exception as e:
    print(f"Failed to save files: {e}")

print("Disk space after saving:")
os.system('df -h /kaggle/working/')

# Prediction function
valid_symptoms = X.columns.tolist()
original_symptoms = augmented_df.drop('diseases', axis=1).columns.tolist()

# Manual disease mapping for precautions
disease_mapping = {
    'viral gastroenteritis': 'Gastroenteritis',
    'influenza': 'Influenza',
    'common cold': 'Common Cold',
    'panic attack': None,
    'hyperemesis gravidarum': 'Gastroenteritis',
    'abscess of the pharynx': 'Common Cold'
}

def match_symptoms(user_symptoms, valid_symptoms, original_symptoms, threshold=85):
    matched_symptoms = []
    for symptom in user_symptoms:
        match = process.extractOne(symptom, original_symptoms, score_cutoff=threshold)
        if match:
            matched_symptom = sanitize_name(match[0])
            matched_symptoms.append(matched_symptom)
            print(f"Matched '{symptom}' to '{match[0]}' (sanitized: '{matched_symptom}', score: {match[1]})")
        else:
            print(f"Warning: No match found for symptom '{symptom}'")
    return matched_symptoms

def predict_disease_and_precautions(symptoms_list):
    matched_symptoms = match_symptoms(symptoms_list, valid_symptoms, original_symptoms)
    if not matched_symptoms:
        return {'Predicted Disease': 'None', 'Precautions': ['No valid symptoms provided.'], 'Confidence': 0.0, 'Top Predictions': []}
    
    symptom_vector = pd.Series(0, index=valid_symptoms)
    for symptom in matched_symptoms:
        if symptom in symptom_vector.index:
            symptom_vector[symptom] = 1
        else:
            print(f"Warning: Sanitized symptom '{symptom}' not in feature set")
    
    symptom_vector = symptom_vector.values.reshape(1, -1)
    
    predicted_label = model.predict(symptom_vector)[0]
    probabilities = model.predict_proba(symptom_vector)[0]
    confidence = probabilities[predicted_label]
    predicted_disease = label_encoder.inverse_transform([predicted_label])[0]
    
    top_indices = np.argsort(probabilities)[-3:][::-1]
    top_predictions = [
        (label_encoder.inverse_transform([idx])[0], probabilities[idx])
        for idx in top_indices
    ]
    
    precaution_diseases = precaution_df['Disease'].tolist()
    top_matches = process.extract(predicted_disease, precaution_diseases, limit=5)
    print(f"Top 5 precaution matches for '{predicted_disease}':")
    for match in top_matches:
        print(f"  - {match[0]} (score: {match[1]})")
    
    # Use manual mapping if available
    matched_disease = disease_mapping.get(predicted_disease, None)
    if matched_disease:
        print(f"Used manual mapping: '{predicted_disease}' → '{matched_disease}'")
    else:
        match = process.extractOne(predicted_disease, precaution_diseases, score_cutoff=60)
        if match:
            matched_disease = match[0]
            print(f"Matched disease '{predicted_disease}' to '{matched_disease}' (score: {match[1]})")
        else:
            matched_disease = predicted_disease
            print(f"No precaution match found for '{predicted_disease}'")
    
    precautions = precaution_df[precaution_df['Disease'] == matched_disease]
    precaution_list = precautions.iloc[0][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].dropna().tolist() if not precautions.empty else ["No precautions found for this disease."]
    
    return {
        'Predicted Disease': predicted_disease,
        'Precautions': precaution_list,
        'Confidence': confidence,
        'Top Predictions': top_predictions
    }

# Test predictions
test_symptoms = [
    ['vomiting', 'diarrhea', 'weakness'],
    ['headache', 'fever', 'cough'],
    ['anxiety and nervousness', 'shortness of breath', 'chest tightness']
]

for symptoms in test_symptoms:
    print(f"\nTesting symptoms: {symptoms}")
    result = predict_disease_and_precautions(symptoms)
    print(f"Predicted Disease: {result['Predicted Disease']}")
    print(f"Confidence: {result['Confidence']:.2f}")
    print("Top 3 Predictions:")
    for disease, prob in result['Top Predictions']:
        print(f"  - {disease}: {prob:.2f}")
    print("Precautions:")
    for i, precaution in enumerate(result['Precautions'], 1):
        print(f"{i}. {precaution}")

print(f"Total runtime: {(time.time() - start_time)/60:.2f} minutes")

Ensured /kaggle/working/ directory exists
Disk space before training:
Filesystem      Size  Used Avail Use% Mounted on
/dev/loop1       20G  2.2M   20G   1% /kaggle/working
Number of unique diseases after filtering: 677
Dataset size: (246512, 378)
Selected 146 features after variance thresholding
Test Set Accuracy: 0.75
Model and Label Encoder saved to /kaggle/working/
Confirmed: model_xgboost_v3.joblib saved
Confirmed: label_encoder_xgboost_v3.joblib saved
Disk space after saving:
Filesystem      Size  Used Avail Use% Mounted on
/dev/loop1       20G  358M   20G   2% /kaggle/working

Testing symptoms: ['vomiting', 'diarrhea', 'weakness']
Matched 'vomiting' to 'vomiting' (sanitized: 'vomiting', score: 100)
Matched 'diarrhea' to 'diarrhea' (sanitized: 'diarrhea', score: 100)
Matched 'weakness' to 'weakness' (sanitized: 'weakness', score: 100)
Top 5 precaution matches for 'gastrointestinal hemorrhage':
  - Gastroenteritis (score: 66)
  - GERD (score: 60)
  - Paralysis (brain hemorrhage) (